### Importing Python libraries and intializing variables

In [0]:
# Imports
from google.colab import auth
from google.colab import drive
auth.authenticate_user()

import gspread
from oauth2client.client import GoogleCredentials

import pandas as pd
import numpy as np
import os, pickle, re, json

In [2]:
# Connecting to Google Drive
drive.mount('/content/drive')
data_pth = '/content/drive/My Drive/chatterine/data'
annotated_pth = os.path.join(data_pth, 'test_data.csv')
df_pth = os.path.join(data_pth, 'df_reduced.csv')
new_obs_pth = os.path.join(data_pth, 'obs_from_website.csv')
model_pth = os.path.join(os.path.dirname(data_pth), 'models')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
def numberToLetters(q):
  """
  Function to identify the cells that need to be populated in the Google sheet 
  to be populated with data fetched from somewhere else.
  """
  q -= 1
  result = ''
  while q >= 0:
      remain = q % 26
      result = chr(remain+65) + result;
      q = q//26 - 1
  return result

### Loading the data set to whose labels we want to add more utterances representing what someone might say when chatting to the bot


In [0]:
gc = gspread.authorize(GoogleCredentials.get_application_default())
sh = gc.open('data')

try:
  ann_by_target = sh.worksheet("2annotate_by_labels")
except:
  ann_by_target = sh.add_worksheet(title="2annotate_by_labels", rows="5000", cols="20")  
  

try:
  complete_sheet = sh.worksheet("annotated")
except:
  complete_sheet = None

In [5]:
utt_sheet = sh.worksheet("MODIFY_CONTENT_HERE")
rows = utt_sheet.get_all_values()
df = pd.DataFrame.from_records(rows[1:])
df.columns = rows[0]
# Removing observations with blank spaces
df.dropna(inplace = True)
df.drop_duplicates(inplace = True)
df = df[(df.utterance.str.len() > 1)]
df = df[(df.intent.str.len() > 1)]
df = df.assign(target=df[['topic_level_0', 'topic_level_1', 'intent']].apply(lambda x: '__'.join(x), axis=1))
df.reset_index(inplace = True, drop = True)
df_utter = df[['target', 'utterance']]
df_utter.to_csv(df_pth)

print(df_utter.shape)
df_utter.head()

(2554, 2)


,target,utterance
0,about_me_personally__my_past__DSR,has she studied data science?
1,about_me_personally__my_past__DSR,I want to know where did rita study to become ...
2,about_me_personally__my_past__DSR,what can you tell about drs?
3,about_me_personally__my_past__DSR,Who enocouraged her to learn machine learning?
4,about_me_personally__my_past__DSR,who helped rita become a data scientist?


### Retrieving the already annotated data contained in the  'annotated' sheet and adding it to the primary data set, containing all the rest of the observations


In [6]:
if complete_sheet and bool(complete_sheet.get_all_values()):
  try:
    label = complete_sheet.col_values(1)
    phrases = complete_sheet.col_values(2)
    df_complete = pd.DataFrame({'target': label[1:], 'utterance': phrases[1:]})
    df_complete.dropna(inplace = True)
    df_complete = df_complete[(df_complete.utterance.str.len() > 1)]
    df_complete.drop_duplicates(inplace = True)
    df_complete.reset_index(inplace = True, drop = True)
    df_complete.to_csv(annotated_pth)
    df_complete.head()
  except:
    df_complete = pd.DataFrame()

else:
  df_complete = pd.DataFrame()

df_complete.head()

,target,utterance
0,about_me_personally__my_past__career_switch_in...,Does she have a mathematical background?
1,about_me_personally__my_past__career_switch_in...,"That's interesting, but I was thinking more ab..."
2,about_me_personally__my_past__career_switch_in...,Does she regret her career switch?
3,about_me_personally__my_past__career_switch_in...,Does she miss her work as a translator?
4,about_me_personally__my_past__career_switch_in...,has rita studied conversation design


In [7]:
df1 = df_utter.append(df_complete, ignore_index=True)
df1.drop_duplicates(inplace=True)
df1.dropna(inplace = True)
df1.sort_values(by = ['target'], inplace = True)
df1.reset_index(inplace = True, drop = True)

print(df1.shape)
df1.head()

(2554, 2)


,target,utterance
0,about_me_personally__my_past__DSR,has she studied data science?
1,about_me_personally__my_past__DSR,where did she learn nlp?
2,about_me_personally__my_past__DSR,wat did rita learn at DSR?
3,about_me_personally__my_past__DSR,what about telling me about dsr
4,about_me_personally__my_past__DSR,did she study at a dsr bootcamp?


### Promting you to decide how much you can afford to annotate and what type of labels you want to add utterances to

In [8]:
# You will be prompted to choose between exporting for annotation to a Google Sheet
# a dataset with the intents that had more and less phrases assigned to them.
utt_x_label = input("""
Choose exporting for annotation to a Google Sheet a dataset 
with the 6 intents that have more phrases assigned to them and the one 
with the ones that have less assigned to them.

Press 1 for the first option and 2 for the second.

""")

if re.match(r'1', utt_x_label):
  props = df1.target.value_counts()[:6].index.tolist()
elif re.match(r'2', utt_x_label):
  props = df1.target.value_counts()[::-1][:6].index.tolist()
else:
  raise Exception("""
  I didn't get that last bit, sorry. 
  Could you please rerun the cell and enter either '1' or '2' 
  into the empty field that is displayed?
  """)

df2ann_sh = df1[df1.target.isin(props)]
df2ann_sh.target.value_counts()


Choose exporting for annotation to a Google Sheet a dataset 
with the 6 intents that have more phrases assigned to them and the one 
with the ones that have less assigned to them.

Press 1 for the first option and 2 for the second.

2


about_the_bot__as_product__cookies              44
about_me_personally__my_past__volunteer_work    43
about_me_personally__my_past__as_writer         43
about_my_work__as_data_person__general_IT       43
about_my_work__as_data_person__opinion_on_AI    41
about_the_bot__as_product__suggestion           41
Name: target, dtype: int64

In [9]:
# Minimum number of phrases an intent should have assigned to it 
# (it sould ideally be higher than 8)
try:
  n_phrases = int(input("""
  What is the minimum amount of phrases you want each intent to have assigned to it?
  
  Enter a round number, ideally not lower than 8, or anything other than a number 
  for the default option, which is 20:

  """))
except:
  n_phrases = 20


  What is the minimum amount of phrases you want each intent to have assigned to it?
  
  Enter a round number, ideally not lower than 8, or anything other than a number 
  for the default option, which is 20:

  50


In [10]:
# I want to have 20 observations per intent, some of which will already have an associated phrase, while the rest won't.
n = 0
df2ann = df2ann_sh.copy(deep = 'True')
for i in range(1, df2ann_sh.shape[0]):
  if (df2ann_sh.iloc[i].target != df2ann_sh.iloc[i-1].target) or (i == len(df)-1):
    while n <= n_phrases-2:
      df2ann = df2ann.append({'target': df2ann_sh.iloc[i-1].target, 'utterance': ''}, ignore_index=True)
      n += 1
  else:
    n += 1
    continue
  n = 0
  
df2ann.sort_values(by = ['target'], inplace = True)
df2ann.reset_index(inplace = True, drop = True)
print("The dataframe with the same number of rows per intent, to be populated with utterances in a Google Sheet, has {0} rows.".format(df2ann.shape[0]))

The dataframe with the same number of rows per intent, to be populated with utterances in a Google Sheet, has 291 rows.


### Sending the chunk of the data set to be annotated to the sheet where one can add observations to labels, called '2annotate_by_labels'

In [11]:
# Writing the names of the columns of the dataset to be exported to a Google sheet in the cells where they belong
columns = df2ann.columns.values.tolist()
cell_list = ann_by_target.range('A1:'+numberToLetters(len(columns))+'1')
for cell in cell_list:
  val = columns[cell.col-1]
  cell.value = val
ann_by_target.update_cells(cell_list)

{'spreadsheetId': '1e53IDkntjo_JGbziIvSk4K0_-A2pnGu8KjlMEE9iOX8',
 'updatedCells': 2,
 'updatedColumns': 2,
 'updatedRange': "'2annotate_by_labels'!A1:B1",
 'updatedRows': 1}

In [12]:
# Filling out the Google sheet with the data containing the filtered out user inputs and the archetypes they got automatically assigned to.
num_lines, num_columns = df2ann.shape
cell_list = ann_by_target.range('A2:'+numberToLetters(num_columns)+str(num_lines+1))
for cell in cell_list:
  val = df2ann.iloc[cell.row-2, cell.col-1]
  cell.value = val
ann_by_target.update_cells(cell_list)

{'spreadsheetId': '1e53IDkntjo_JGbziIvSk4K0_-A2pnGu8KjlMEE9iOX8',
 'updatedCells': 582,
 'updatedColumns': 2,
 'updatedRange': "'2annotate_by_labels'!A2:B292",
 'updatedRows': 291}

### Adding the new observations from the interactions people have had with the bot to the primary data set, so that they can be annotated in the sheet called 'MODIFY_CONTENT_HERE'

In [13]:
df_new_obs = pd.read_csv(new_obs_pth, index_col = 0)
df_new_obs.drop_duplicates(subset='utterance', inplace = True)

df_new_obs.tail(15)

,intent_pred,intent_score,utterance,topic_level_0_pred,topic_level_0_score,topic_level_1_pred,topic_level_1_score
119,university_experience,18.484791,What have you learned in your work as a lingu...,NaN,NaN,NaN,NaN
120,hobbies,24.702557,What do you think that brings you your knowle...,NaN,NaN,NaN,NaN
121,translation_advice,21.017148,That the machine can bring a change to reality?,NaN,NaN,NaN,NaN
124,NaN,NaN,Ok...,about_the_bot,68.854919,NaN,NaN
125,history,49.949702,What's your name,NaN,NaN,NaN,NaN
127,NaN,NaN,My name is Nils.,about_the_bot,78.742957,NaN,NaN
128,personal_opinion,37.284669,What is Ritas favourite color?,NaN,NaN,NaN,NaN
129,university_experience,48.177501,Which universities did Rita visit?,NaN,NaN,NaN,NaN
130,NaN,NaN,Cool/,about_the_bot,90.466938,NaN,NaN
131,NaN,NaN,:D,about_the_bot,56.731687,NaN,NaN


In [14]:
df_news = pd.concat([pd.DataFrame('d__d__d', index=range(df_new_obs.shape[0]), columns=['target']), df_new_obs['utterance']], axis = 1)
df_news.head()

,target,utterance
0,d__d__d,how well does rita speak arabic?
1,d__d__d,te fijas en si estoy hablando en ingles?
2,d__d__d,I thinks this won't work.
3,d__d__d,you speak french?
4,d__d__d,my email is pepe.juanito.rodriguez@gmail.com


In [15]:
df1.head()

,target,utterance
0,about_me_personally__my_past__DSR,has she studied data science?
1,about_me_personally__my_past__DSR,where did she learn nlp?
2,about_me_personally__my_past__DSR,wat did rita learn at DSR?
3,about_me_personally__my_past__DSR,what about telling me about dsr
4,about_me_personally__my_past__DSR,did she study at a dsr bootcamp?


In [16]:
df2 = pd.concat([df1, df_news], axis=0)
df2.drop_duplicates(inplace = True)
df2.drop_duplicates(subset='utterance', inplace = True)

df2.dropna(inplace = True)
df2[['topic_level_0', 'topic_level_1', 'intent']] = df2.target.str.split(pat='__', expand=True)
df2.drop('target', axis=1, inplace= True)
df2.reset_index(inplace=True, drop=True)
print(df2.shape)
df2.tail()

(2561, 4)


,utterance,topic_level_0,topic_level_1,intent
2556,"""Your sentences are correct but that's your on...",d,d,d
2557,"""this is sad to read - """"could not care less""""...",d,d,d
2558,"""I do not enjoy this experience so far, but I ...",d,d,d
2559,is she a friend of arjun?,d,d,d
2560,I want to hire Rita!,d,d,d


In [23]:
# Filling out the Google sheet with the data containing the filtered out user inputs and the archetypes they got automatically assigned to.
num_lines, num_columns = df2.shape
columns = df2.columns.values.tolist()
utt_sheet = sh.worksheet("MODIFY_CONTENT_HERE")

cell_list = utt_sheet.range('A1:'+numberToLetters(num_columns)+'1')
for cell in cell_list:
  val = columns[cell.col-1]
  cell.value = val
utt_sheet.update_cells(cell_list)

cell_list = utt_sheet.range('A2:'+numberToLetters(num_columns)+str(num_lines+1))
for cell in cell_list:
  val = df2.iloc[cell.row-2, cell.col-1]
  cell.value = val
utt_sheet.update_cells(cell_list)

{'spreadsheetId': '1e53IDkntjo_JGbziIvSk4K0_-A2pnGu8KjlMEE9iOX8',
 'updatedCells': 10244,
 'updatedColumns': 4,
 'updatedRange': 'MODIFY_CONTENT_HERE!A2:D2562',
 'updatedRows': 2561}

### Inspecting feature importance with eli5 to get a sense of what is missing that we believe should be representative of what a given intent stands for

In [18]:
df_utter.sample(5)

,target,utterance
2260,about_the_bot__as_product__cookies,how do you process my personal information?
2330,about_the_bot__as_product__elaborate_request,which terms?
1863,about_the_bot__as_character__history,Hi rita!
2044,about_the_bot__as_character__praise_compliment...,can you talk dirty to me?
282,about_me_personally__my_past__before_18,Where is she from?


In [19]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

skf = StratifiedKFold(n_splits=4, random_state=43, shuffle=True)
vec = TfidfVectorizer(analyzer='word', ngram_range=(1, 5))

X = vec.fit_transform(df_utter.utterance)

y_preds = []
y_trues = []
i = 0
for tr, te in skf.split(df_utter.utterance, df_utter.target):
  i += 1
  print("\nThe model is now training with the data from the cross validation set number {} ...\n".format(i))
  clf = LogisticRegression(C=5000, solver='lbfgs', multi_class='multinomial', verbose = 2)
  clf.fit(X[tr], df_utter.iloc[tr].target)
  y_preds.append(clf.predict(X[te]))
  y_trues.append(df_utter.iloc[te].target)


The model is now training with the data from the cross validation set number 1 ...



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   30.6s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   30.6s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.



The model is now training with the data from the cross validation set number 2 ...



/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   31.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   31.1s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.



The model is now training with the data from the cross validation set number 3 ...



/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   31.3s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   31.3s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.



The model is now training with the data from the cross validation set number 4 ...



/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   30.9s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   30.9s finished


In [0]:
# Installing the visualisation package only if needed
try:
  import eli5
except:
  !pip install -q eli5
  import eli5

In [21]:
print("""
The table below shows you which sequence of words or characters seemed most relevant
(in green) and least relevant (in red) to the model to predict which phrase belonged to 
which semantic category (intent/archetype).
""")
eli5.show_weights(clf, vec=vec, top=15)


The table below shows you which sequence of words or characters seemed most relevant
(in green) and least relevant (in red) to the model to predict which phrase belonged to 
which semantic category (intent/archetype).



In [22]:
# Visualizing it on one observation
sent = input("""
What else would you expect someone trying to chat with your bot to say? 

""")

print("""
The labels that, according to the model, had a higher probability of 
being the correct ones, appear listed first.
""")
eli5.show_prediction(clf, doc=sent, vec=vec, top_targets= 10)

KeyboardInterrupt: ignored